In [ ]:
!python -V

In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_parquet('data/yellow_tripdata_2022-01.parquet')

In [ ]:
print(df.shape)

In [ ]:
# df = pd.read_parquet('./data/green_tripdata_2021-01.parquet')

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)



In [ ]:
df.duration.std()

In [ ]:
print(df.shape[0])

In [ ]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [ ]:
print(df.shape[0])

In [ ]:
2421440/2463931

In [ ]:
# train_dicts = df[categorical + numerical].to_dict(orient='records')
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
df.shape

In [ ]:
X_train.shape

In [ ]:
# sns.distplot(y_pred, label='prediction')
# sns.distplot(y_train, label='actual')

# plt.legend()
# plt.show()

## Read dataframe

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
# df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
# df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [ ]:
# df_train.columns

In [ ]:
# len(df_train), len(df_val)
print(df_train.shape)
print(df_val.shape)

## Train model

In [ ]:
# df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
# df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
# categorical = ['PU_DO'] 
categorical = ['PULocationID','DOLocationID']
# numerical = ['trip_distance']

dv = DictVectorizer()

# train_dicts = df_train[categorical + numerical].to_dict(orient='records')
train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# val_dicts = df_val[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
X_train.shape

(2421440, 515)

In [9]:
# train_dicts = df[categorical].to_dict(orient='records')

# dv = DictVectorizer()
# X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)



LinearRegression()

## Evaluating model

In [10]:
# training RMSE
# y_train_pred = lr.predict(X_train)
# mean_squared_error(y_train, y_train_pred, squared=False)

y_train_pred = lr.predict(X_train)
print('Train RMSE :'+str(mean_squared_error(y_train, y_train_pred, squared=False)))

Train RMSE :6.98619073816116


In [11]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
# lr = Lasso(0.01)
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_val)

# mean_squared_error(y_val, y_pred, squared=False)

In [12]:
y_val = df_val[target].values

y_val_pred = lr.predict(X_val)
print('Validation RMSE :'+str(mean_squared_error(y_val, y_val_pred, squared=False)))

Validation RMSE :7.786406385475696
